In [27]:
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper
api_wrapper_wiki=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)


In [28]:
api_wrapper_wiki=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
api_wrapper_arxiv=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
wiki.name

'wikipedia'

In [29]:
api_wrapper_arxiv=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
print(arxiv.name)

arxiv


In [30]:
tools=[wiki,arxiv]

In [31]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.llms import Ollama
import os

In [32]:

from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [33]:
loader=WebBaseLoader("https://docs.smith.langchain.com/")

In [34]:
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(docs,embeddings)
retriever=vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002322847C890>, search_kwargs={})

In [35]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith_search","seaerch any information about langsmith")

In [36]:
retriever_tool.name

'langsmith_search'

In [37]:
tools=[wiki,arxiv,retriever_tool]

In [42]:
from langchain_groq import ChatGroq
groq_api_key=os.getenv("GROQ_API_KEY")
llm=ChatGroq(groq_api_key="",model_name="Llama3-8b-8192")

In [43]:
from langchain import hub
prompt=hub.pull("hwchase17/openai-functions-agent")

c:\Users\chafl\miniconda3\envs\hello\Lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [44]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [45]:
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [46]:
from langchain.agents import AgentExecutor
agent_exe=AgentExecutor(agent=agent,tools=tools,verbose=True)

In [49]:
agent_exe.invoke({"input":"what is langsmith"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Langsmith'}`


No good Wikipedia Search Result was foundLet's try another tool. <tool-use>{"tool_calls":[{"id":"pending","type":"function","function":{"name":"arxiv"},"parameters":{"query":"langsmith"}}]}</tool-use>

> Finished chain.


{'input': 'what is langsmith',
 'output': 'Let\'s try another tool. <tool-use>{"tool_calls":[{"id":"pending","type":"function","function":{"name":"arxiv"},"parameters":{"query":"langsmith"}}]}</tool-use>'}

In [ ]:
import streamlit as st 
from langchain_groq import ChatGroq
from langchain_community.utilities import ArxivAPIWrapper,WikipediaAPIWrapper
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun,DuckDuckGoSearchRun
from langchain.agents import initialize_agent,AgentType
from langchain.callbacks import StreamlitCallbackHandler
import os
from dotenv import load_dotenv
load_dotenv()

api_wrapper_wiki=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
api_wrapper_arxiv=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
search=DuckDuckGoSearchRun(name="Search")
st.sidebar.title("Settings")
api_key=st.sidebar.text_input("enter your api key",type="password")



In [52]:
st.title("Lanagchain chat woth search")
"""
in this ecample we using streamlitcallback hadler to display thoughts and actions
try more langachain strreamlit example

"""


if "messages" not in st.session_state:
    st.session_state["messages"]=[
        {"role":"assistant","content":"hi im a chatbot who can search the web "}
    ]

NameError: name 'st' is not defined

In [53]:
for msg in st.session_state.messages:
    st.chat_message(msg["role".write(msg['content'])])

NameError: name 'st' is not defined

In [54]:
if prompt:=st.chat_input(placeholder="what is machine learning?"):
    st.session_state.messages.append({"role":"user","content":prompt})
    st.chat_message("user").write(prompt)
    llm=ChatGroq(groq_api_key="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",model_name="Llama3-8b-8192",streaming=True)
    tools=[search,wiki,arxiv]
    
    search_agent=initialize_agent(tools,llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,handling_parsing_errors=True)
    
    with st.chat_message("assistant"):
        st_cb=StreamlitCallbackHandler(st.container(),expand_new_thoughts=False)
        res=search_agent.run(st.session_state.messages,callbacks=[st_cb])
        st.session_state.messages.append({'role':'assistant','content':res})
    

NameError: name 'st' is not defined